Resources used:
- Instalation: https://medium.com/@oleg.tarasov/building-fasttext-python-wrapper-from-source-under-windows-68e693a68cbb
https://fasttext.cc/docs/en/supervised-tutorial.html

https://ravindraelicherla.medium.com/build-your-own-text-classification-in-less-than-25-lines-of-code-using-fasttext-dae7229f80f9

https://www.youtube.com/watch?v=TmfIcqxMdbc

https://medium.com/@oleg.tarasov/building-fasttext-python-wrapper-from-source-under-windows-68e693a68cbb 

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    
https://www.youtube.com/watch?v=tAxrlAVw-Tk&t=43s



# fasttext



In [2]:
import fasttext
import pandas as pd
import numpy as np
import nltk

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

df = pd.read_csv("final_final_file.csv", 
                 sep = '\t')

In [3]:
bin_label_list = []
temp_list = df['sentiment'].to_list()
for i in temp_list:
    if (i == 'Not distored'):
        bin_label_list.append('Not distored') #'Not distored'
    else:
        bin_label_list.append('Distored') #'Distored'

df['bin_sentiment'] = bin_label_list

In [4]:
df.head()

,text,sentiment,bin_sentiment
0,something that i have been noticing recently i...,Catastrophizing,Distored
1,i went on vacation and just got back yesterday...,Catastrophizing,Distored
2,i've had crippling depression since i started ...,Jumping to conclusion,Distored
3,i feel like confusion is a huge part to depres...,Not distored,Not distored
4,i skipped both of my classes yesterday. i had ...,Not distored,Not distored


In [5]:
#https://www.kdnuggets.com/2020/05/dataset-splitting-best-practices-python.html

X, y = df['text'], df['bin_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size = 0.7, 
                                                    random_state = 42,
                                                    stratify = y)

In [6]:
new_df_train = pd.DataFrame()
new_df_train['label'] = y_train
new_df_train['texts'] = X_train

new_df_train['label']=['__label__'+s.replace(' or ', '$').replace(', or ','$').replace(',','$').replace(' ','_').replace(',','__label__').replace('$$','$').replace('$',' __label__').replace('___','__') for s in new_df_train['label']]
new_df_train['label']

new_df_train['texts']= new_df_train['texts'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

new_df_train.to_csv("cog_bin_train.txt", sep='\t', index = False)

In [7]:
new_df_test = pd.DataFrame()
new_df_test['label'] = y_test
new_df_test['texts'] = X_test

new_df_test['label']=['__label__'+s.replace(' or ', '$').replace(', or ','$').replace(',','$').replace(' ','_').replace(',','__label__').replace('$$','$').replace('$',' __label__').replace('___','__') for s in new_df_test['label']]
new_df_test['label']

new_df_test['texts']= new_df_test['texts'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

new_df_test.to_csv("cog__bin_test.txt", sep='\t', index = False)

In [98]:
ft_model = fasttext.train_supervised(input = "cog_bin_train.txt", lr=0.69, epoch = 5)

In [99]:
ft_model.test("cog__bin_test.txt")

(580, 0.7189655172413794, 0.7189655172413794)

In [100]:
temp_list = new_df_test["texts"].to_list()
temp = pd.DataFrame()
temp["predicted"] = new_df_test["texts"].apply(lambda x: ft_model.predict(x)[0][0])

from sklearn.metrics import confusion_matrix

confusion_matrix(new_df_test['label'], temp["predicted"])

array([[287,  70],
       [ 95, 128]], dtype=int64)

In [101]:
print(classification_report(new_df_test['label'], temp["predicted"], digits = 4))

cm = confusion_matrix(new_df_test['label'], temp["predicted"])
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                       precision    recall  f1-score   support

    __label__Distored     0.7513    0.8039    0.7767       357
__label__Not_distored     0.6465    0.5740    0.6081       223

             accuracy                         0.7155       580
            macro avg     0.6989    0.6890    0.6924       580
         weighted avg     0.7110    0.7155    0.7119       580

[0.80392157 0.57399103]


# Multiclass

In [61]:
mult_df = df[df['sentiment'] != 'Not distored']

In [62]:
#https://www.kdnuggets.com/2020/05/dataset-splitting-best-practices-python.html

X, y = mult_df['text'], mult_df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size = 0.7, 
                                                    random_state = 42,
                                                    stratify = y)

In [63]:
new_df_train = pd.DataFrame()
new_df_train['label'] = y_train
new_df_train['texts'] = X_train

new_df_train['label']=['__label__'+s.replace(' or ', '$').replace(', or ','$').replace(',','$').replace(' ','_').replace(',','__label__').replace('$$','$').replace('$',' __label__').replace('___','__') for s in new_df_train['label']]
new_df_train['label']

new_df_train['texts']= new_df_train['texts'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

new_df_train.to_csv("cog_multi_train.txt", sep='\t', index = False)

In [64]:
new_df_test = pd.DataFrame()
new_df_test['label'] = y_test
new_df_test['texts'] = X_test

new_df_test['label']=['__label__'+s.replace(' or ', '$').replace(', or ','$').replace(',','$').replace(' ','_').replace(',','__label__').replace('$$','$').replace('$',' __label__').replace('___','__') for s in new_df_test['label']]
new_df_test['label']

new_df_test['texts']= new_df_test['texts'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

new_df_test.to_csv("cog__multi_test.txt", sep='\t', index = False)

In [112]:
ft_model = fasttext.train_supervised(input = "cog_multi_train.txt", lr = 1,
                                     epoch = 5)

ft_model.test("cog__multi_test.txt")

(357, 0.27450980392156865, 0.27450980392156865)

In [113]:
temp_list = new_df_test["texts"].to_list()
temp = pd.DataFrame()
temp["predicted"] = new_df_test["texts"].apply(lambda x: ft_model.predict(x)[0][0])

from sklearn.metrics import confusion_matrix

confusion_matrix(new_df_test['label'], temp["predicted"])

array([[ 9,  0,  0, 19, 18,  2,  0,  3,  0],
       [ 5,  0,  0,  5, 10,  3,  0,  6,  0],
       [ 1,  0,  0, 15,  9,  2,  0,  7,  0],
       [ 3,  0,  0, 31, 18,  1,  0,  0,  1],
       [ 4,  0,  0, 21, 25,  4,  0,  1,  2],
       [ 0,  0,  0,  9, 10, 13,  0,  1,  2],
       [ 4,  0,  0, 16, 17,  0,  0,  4,  0],
       [ 4,  0,  0,  2, 14,  1,  1,  8,  1],
       [ 0,  0,  0,  7,  3,  2,  0,  2, 11]], dtype=int64)

In [114]:
print(classification_report(new_df_test['label'], temp["predicted"], digits = 4))

                                     precision    recall  f1-score   support

 __label__Black_and_white_thinking.     0.3000    0.1765    0.2222        51
           __label__Catastrophizing     0.0000    0.0000    0.0000        29
__label__Disqualifying_the_positive     0.0000    0.0000    0.0000        34
       __label__Emotional_reasoning     0.2480    0.5741    0.3464        54
     __label__Jumping_to_conclusion     0.2016    0.4386    0.2762        57
                  __label__Labeling     0.4643    0.3714    0.4127        35
       __label__Over-generalization     0.0000    0.0000    0.0000        41
           __label__Personalization     0.2500    0.2581    0.2540        31
         __label__Should_statements     0.6471    0.4400    0.5238        25

                           accuracy                         0.2717       357
                          macro avg     0.2346    0.2510    0.2261       357
                       weighted avg     0.2251    0.2717    0.2274       3